### Smart Personal Finance Analyzer - AI Capstone Project

In [5]:
import csv
from datetime import datetime

transactions = []
ERROR_LOG_FILE = 'errors.txt'

def _log_error(message):
    with open(ERROR_LOG_FILE, 'a') as f:
        f.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

def load_transactions(filename='financial_transactions_short.csv'):
    global transactions    

    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                processed_row = row.copy()

                date_str = processed_row.get('date', '').strip()
                parsed_date = None
                date_formats = ["%Y-%m-%d", "%d-%m-%Y", "%m/%d/%Y"]

                if date_str:
                    for fmt in date_formats:
                        try:
                            parsed_date = datetime.strptime(date_str, fmt)
                            break
                        except ValueError:
                            continue
                    if parsed_date:
                        processed_row['date'] = parsed_date
                    else:
                        _log_error(f"Error: Skipping transaction {processed_row.get('transaction_id', 'N/A')}. Invalid date format '{date_str}.")
                        processed_row['date'] = None

                    try:
                        amount_str = processed_row.get('amount', '').strip()
                        new_amount = 0.0
                        if not amount_str:
                            _log_error(f"Warning: Empty amount found for transaction {processed_row.get('transaction_id', 'N/A')}. Setting to 0.0.")
                        else:
                            new_amount = float(amount_str)
                        transaction_type = processed_row.get('type', '').strip().lower()
                        if transaction_type == "debit":
                            processed_row['amount'] = new_amount * -1
                        elif transaction_type == 'credit':
                            processed_row['amount'] = new_amount
                        else:
                            processed_row['amount'] - new_amount
                            if transaction_type:
                                _log_error(f"Warning: Unrecognizable type '{transaction_type} for transaction {processed_row.get('transaction_id', 'N/A')}. Amount stored as is.")
                    except (ValueError, TypeError):
                        _log_error(f"Error: Could not convert amount to float in transaction {processed_row.get('transaction_id', 'N/A')}. Invalid amount '{processed_row.get('amount', '')}'.")
                        processed_row['amount'] = 0.0

                    transactions.append(processed_row)

    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        _log_error(f"Error: The file '{filename}' was not found.")
    except Exception as e:
        print(f"An unexpected error occurred during file loading: {e}")
        _log_error(f"An unexpected error occurred during file loading: {e}")
    
    print(f"There are {len(transactions)} transactions.")
    print(f"Transaction processing complete. Check {ERROR_LOG_FILE} for any logged issues.")

def add_transactions(transactions):
    load_transactions()
    print(transactions)
    max_id = 0
    
    for transaction in transactions:
        if 'transaction_id' in transaction:
            try:
                current_id = int(transaction['transaction_id'])
                if current_id > max_id:
                    max_id = current_id
            except (ValueError, TypeError):
                print(f"Warning: transaction_id '{transaction['transaction_id']}' is not a valid number. Skipping.")
        else:
            print("Warning: 'transaction_id' key not found in transaction.")
   
    print("--- Add New Transaction ---")
    
    transaction_id = max_id + 1

    while True:
        date_str = input("Enter date (YYYY-MM-DD): ").strip()
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

    customer_id = input("Enter customer ID: ")

    while True:
        amount_str = input("Enter amount: ").strip()
        try:
            amount = float(amount_str)
            break
        except ValueError:
            print("Invalid amount. Please enter numerical value.")

    type = input("Enter type (credit/debit/transfer): ").strip()
    if not type:
        print("Transaction type cannot be empty. Aborting...")
        return
      
    description = input("Enter description for transaction: ").strip()

    try:
        new_transaction = {'transaction_id': transaction_id, 'date': date_str, 'customer_id': customer_id, 'amount': amount, 'type': type, 'description': description}
        transactions.append(new_transaction)
        print(f"Enter date (YYYY-MM-DD): {date_str}")
        print(f"Enter customer ID: {customer_id}")
        print(f"Enter amount: {amount}")
        print(f"Enter type (credit/debit/transfer): {type}")
        print(f"Enter description: {description}")
        print(f"Transaction added!")
        print(transactions[-1])
    except IOError as e:
        print(f"Unable to add transaction.")

add_transactions(transactions)

There are 20 transactions.
Transaction processing complete. Check errors.txt for any logged issues.
[{'transaction_id': '1', 'date': datetime.datetime(2020, 10, 26, 0, 0), 'customer_id': '926', 'amount': 6478.39, 'type': 'credit', 'description': 'Expect series shake art again our.'}, {'transaction_id': '2', 'date': datetime.datetime(2020, 1, 8, 0, 0), 'customer_id': '466', 'amount': 1255.95, 'type': 'credit', 'description': 'Each left similar likely coach take.'}, {'transaction_id': '3', 'date': datetime.datetime(2019, 9, 2, 0, 0), 'customer_id': '110', 'amount': -7969.68, 'type': 'debit', 'description': 'Direction wife job pull determine leader move college.'}, {'transaction_id': '4', 'date': datetime.datetime(2020, 12, 2, 0, 0), 'customer_id': '142', 'amount': 2927.41, 'type': 'credit', 'description': 'Agree reveal buy black already.'}, {'transaction_id': '5', 'date': datetime.datetime(2020, 12, 2, 0, 0), 'customer_id': '944', 'amount': -4661.88, 'type': 'debit', 'description': 'Chil

In [29]:
transactions = []

def view_transactions(transactions):
    if not transactions:
        print(f"No transactions to display. Load or add some transactions.")
        return
    
    columns = [('transaction_id', 'ID', 6), ('date', 'Date', 12), ('customer_id', 'Customer', 10),
               ('amount', 'Amount', 12), ('type', 'Type', 10), ('description', 'Description', 40)]
    
    header_parts = []
    separator_parts = []
    for key, display_name, width in columns:
        header_parts.append(f"{display_name:<{width}}")
        separator_parts.append("-" * width)
    print(" | ".join(header_parts))
    print(" | ".join(separator_parts))

    for transaction in transactions:
        row_values = []
        for key, _, width in columns:
            value = transaction.get(key, '')

            if key == 'amount':
                try:
                    value = f"{float(value):.2f}"
                except (ValueError, TypeError):
                    value = "N/A"

            str_value = str(value)

            if len(str_value) > width:
                str_value = str_value[:width - 3] + "..."

            row_values.append(f"{str_value:<{width}}")

        print(" | ".join(row_values))

view_transactions(transactions)

No transactions to display. Load or add some transactions.


### Main Program - Smart Personal Finance Analyzer

In [2]:
import csv
from datetime import datetime
import personal_finance_lib5
transactions = []

def main():
    while True:
        print("\nSmart Personal Finance Analyzer")
        print("1. Load Transactions")
        print("2. Add Transaction")
        print("3. View Transactions")
        print("4. Update Transaction")
        print("5. Delete Transaction")
        print("6. Analyze Finances")
        print("7. Save Transactions")
        print("8. Generate Report")
        print("9. Exit")
        choice = input("Select an option: ")
        # Call functions based on choice
        if choice == '9':
            break
        elif choice == '1':
            load_transactions()
        elif choice == '2':
            personal_finance_lib5.add_transactions(transactions)
        elif choice == '3':
            view_transactions(transactions)
        elif choice == '4':
            personal_finance_lib5.delete_transactions(transactions)


main()


Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit

Smart Personal Finance Analyzer
1. Load Transactions
2. Add Transaction
3. View Transactions
4. Update Transaction
5. Delete Transaction
6. Analyze Finances
7. Save Transactions
8. Generate Report
9. Exit
An unexpected error occurred during file loading: cannot access local variable 'csv' where it is not associated with a value
There are 0 transactions.
Transaction processing complete. Check errors.txt for any logged issues.
--- Add New Transaction ---
Enter date (YYYY-MM-DD): 2025-06-05
Enter customer ID: 359
Enter amount: 162.55
Enter type (credit/debit/transfer): credit
Enter description: test transaction
Transaction added!
{'transaction_id': 1, 'date': '2025-06-05', 'customer_id': '359', 'amount': 162.55, 'type': 'credit', 'description': 'test transaction'}

Smart Personal Finan

NameError: name 'view_transactions' is not defined

In [9]:
import csv
from datetime import datetime
transactions = []

def load_transactions(filename = 'financial_transactions_short.csv'):
    ERROR_LOG_FILE = 'errors.txt'
    try:
        with open(ERROR_LOG_FILE, 'w') as f:
            f.write(f"[{datetime.now()}] --- Transaction Processing Log Started ---\n")
        print(f"Log file '{ERROR_LOG_FILE}' has been created/cleared.")
    except IOError as e:
        print(f"Failed to initialize error log file '{ERROR_LOG_FILE}': {e}")
      
    try:
        with open(filename, 'r', newline='') as file:
            csv_reader = csv.DictReader(file)
            for row in csv_reader:
                transactions.append(row)                
    except FileNotFoundError:
        print(f"Error: The file '{filename}' was not found.")
        return None
    
    for item in transactions:
        date_str = item['date']
        try:
            parsed_date = datetime.strptime(date_str, "%Y-%m-%d")
        except ValueError:
            try:
                parsed_date = datetime.strptime(date_str, "%d-%m-%Y")
            except ValueError:
                try:
                    parsed_date = datetime.strptime(date_str, "%m/%d/%Y")
                except ValueError:
                    parsed_date = None

        if parsed_date:
            item['date'] = parsed_date.strftime("%Y-%m-%d")
        else:
            with open(ERROR_LOG_FILE, 'a') as f:
                f.write(f"Error: Skipping transaction {item['transaction_id']}. Invalid date.\n")

    for item in transactions:
        try:
            if not item['amount'] or item['amount'].strip() =="":
                new_amount = 0.0
                print(f"Warning: Empty amount found for transaction {item['transaction_id']}. Setting to 0.0")
            else:
                new_amount = float(item['amount'])
            
            if item['type'] == "debit":
                item['amount'] = new_amount * -1
            elif item['type'] == "credit":
                item['amount'] = new_amount            
        except ValueError:
            with open(ERROR_LOG_FILE, 'a') as f:
                f.write(f"Error: Could not convert amount to float in transaction {item['transaction_id']}. Invalid amount.")



    print(f"There are {len(transactions)} transactions.")
    print(f"Transaction processing complete. Check {ERROR_LOG_FILE} for any logged issues.")

    return transactions

load_transactions()

def add_transactions(transactions):
    max_id = 0
   
    print("--- Add New Transaction ---")
    
    for transaction in transactions:
        if int(transaction['transaction_id']) > max_id:
            max_id = int(transaction['transaction_id'])
    
    transaction_id = max_id + 1

    while True:
        date_str = input("Enter date (YYYY-MM-DD): ").strip()
        try:
            datetime.strptime(date_str, '%Y-%m-%d')
            break
        except ValueError:
            print("Invalid date format. Please use YYYY-MM-DD.")

    customer_id = input("Enter customer ID: ")

    while True:
        amount_str = input("Enter amount: ").strip()
        try:
            amount = float(amount_str)
            break
        except ValueError:
            print("Invalid amount. Please enter numerical value.")

    type = input("Enter type (credit/debit/transfer): ").strip()
    if not type:
        print("Transaction type cannot be empty. Aborting...")
        return
    else:
        if type == "debit":
            amount = amount * -1
        else:
            amount = amount
            
    description = input("Enter description for transaction: ").strip()

    try:
        new_transaction = {
            'transaction_id': transaction_id,
            'date': date_str,
            'customer_id': customer_id,
            'amount': amount,
            'type': type,  
            'description': description
        }
        transactions.append(new_transaction)
        print(f"Enter date (YYYY-MM-DD): {date_str}")
        print(f"Enter customer ID: {customer_id}")
        print(f"Enter amount: {amount}")
        print(f"Enter type (credit/debit/transfer): {type}")
        print(f"Enter description: {description}")
        print(f"Transaction added!")
        print(transactions[-1])
    except Exception as e: # A more general exception for unexpected issues during transaction creation/append
        print(f"An unexpected error occurred while adding the transaction: {e}")

add_transactions(transactions)

def view_transactions(transactions):
    print("\n--- Viewing Transactions ---")
    columns = [('transaction_id', 'ID', 6), ('date', 'Date', 12), ('customer_id', 'Customer', 10),
               ('amount', 'Amount', 12), ('type', 'Type', 10), ('description', 'Description', 40)]
    
    header_parts = []
    separator_parts = []
    for key, display_name, width in columns:
        header_parts.append(f"{display_name:<{width}}")
        separator_parts.append("-" * width)
    print(" | ".join(header_parts))
    print(" | ".join(separator_parts))

    for transaction in transactions:
        row_values = []
        for key, _, width in columns:
            value = transaction.get(key, '')

            if key == 'amount':
                try:
                    value = f"{float(value):.2f}"
                except (ValueError, TypeError):
                    value = "N/A"

            str_value = str(value)

            if len(str_value) > width:
                str_value = str_value[:width - 3] + "..."

            row_values.append(f"{str_value:<{width}}")

        print(" | ".join(row_values))

view_transactions(transactions)

def update_transactions(transactions):
    print("\n--- Updating Transaction ---")

    if not transactions:
        print("No transactions to update.")
        return
    
    view_transactions(transactions)

    while True:
        try:
            transaction_to_update = int(input("\nEnter the ID of the transaction to update (or 0 to cancel): "))
            if transaction_to_update == 0:
                print("Update cancelled.")
                return
            break
        except ValueError:
            print("Invalid input. Please enter a numerical transaction ID.")

    found = None
    for transaction in transactions:
        if int(transaction.get('transaction_id', 0)) == transaction_to_update:
            found = transaction
            break
    
    if not found:
        print(f"Transaction with ID {transaction_to_update} not found.")
        return
    
    print(f"\nTransaction found. Current details for {transaction_to_update}: ")
    print(f"1. Description: {found.get('description', '')}")
    print(f"2. Type: {found.get('type', '')}")
    print(f"3. Amount: {found.get('amount', 0)}")

    while True:
        field_choice = input("Enter the number of the field to update (1-3): ")

        if field_choice == '0':
            print("Update cancelled.")
            return
        
        if field_choice == '1':
            new_description = input("Enter new description: ").strip()
            found['description'] = new_description
            print("Description updated successfully.")
            break
        elif field_choice == '2':
            while True:
                new_type = input("Enter new type (credit/debit/transfer): ").strip()
                if new_type in ["credit", "debit", "transfer"]:
                    if new_type == "debit":
                        found['type'] = new_type
                        found['amount'] = found['amount'] * -1
                        continue
                    else:
                        found['type'] = new_type
                        print("Type updated successfully.")
                        break
                else:
                    print("Invalid type. Please enter 'credit', 'debit', or 'transfer'.")
            break
        elif field_choice == '3':
            while True:
                try:
                    new_amount = float(input("Enter new amount: ").strip())
                    if found.get('type') == 'debit':
                        found['amount'] = new_amount * -1
                    else:
                        found['amount'] = new_amount
                    print("Amount updated successfully.")
                    break
                except ValueError:
                    print("Invalid amount. Please enter a numerical value.")
            break
        else:
            print("Invalid choice. Please enter 1, 2, 3, or 0 to cancel.")
    
    print("\nUpdated Transaction Details:")
    
update_transactions(transactions)

def delete_transactions(transactions):
    print("\n--- Delete Transaction ---")

    if not transactions:
        print("Cannot delete transactions as no data was loaded.")
        return
    
    print("\nTransactions available for deletion:")

    view_transactions(transactions)

    while True:
        try:
            transaction_to_delete = int(input("\nEnter the ID of the transaction to delete (or 0 to cancel): "))

            if transaction_to_delete == 0:
                print("Deletion cancelled.")
                return
            
            found = False
            for i, transaction in enumerate(transactions):
                if transaction['transaction_id'] == transaction_to_delete:
                    print(f"\nFound transaction with ID {transaction_to_delete}:")
                    print(f"Date: {transaction['date']}, Customer: {transaction['customer_id']}, Amount: {transaction['amount']:.2f}, Type: {transaction['type']}")
                    confirm = input("Are you sure your want to delete this transaction? (yes/no): ").strip().lower()

                    if confirm == 'yes':
                        del transactions[i]
                        print(f"Transaction with ID {transaction_to_delete} deleted successfully.")
                    else:
                        print("Deletion aborted by user.")
                    found = True
                    break
            if not found:
                print(f"Transaction with ID {transaction_to_delete} not found.")
            
            break

        except ValueError:
            print("Invalid input. Please enter a numerical transaction ID.")
        except Exception as e:
            print(f"An unexpcted error occurred: {e}")

delete_transactions(transactions)

Log file 'errors.txt' has been created/cleared.
There are 20 transactions.
Transaction processing complete. Check errors.txt for any logged issues.
--- Add New Transaction ---


Enter date (YYYY-MM-DD): 2025-06-05
Enter customer ID: 485
Enter amount: 125.36
Enter type (credit/debit/transfer): credit
Enter description: test transaction
Transaction added!
{'transaction_id': 21, 'date': '2025-06-05', 'customer_id': '485', 'amount': 125.36, 'type': 'credit', 'description': 'test transaction'}

--- Viewing Transactions ---
ID     | Date         | Customer   | Amount       | Type       | Description                             
------ | ------------ | ---------- | ------------ | ---------- | ----------------------------------------
1      | 2020-10-26   | 926        | 6478.39      | credit     | Expect series shake art again our.      
2      | 2020-01-08   | 466        | 1255.95      | credit     | Each left similar likely coach take.    
3      | 2019-09-02   | 110        | -7969.68     | debit      | Direction wife job pull determine lea...
4      | 2020-12-02   | 142        | 2927.41      | credit     | Agree reveal buy black already.         
5      | 2020-12-0